In [3]:
import sys
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Imputer
import numpy 
from collections import Counter


from IPython.core.display import display


def main():
    training_set = pd.read_csv("train.csv")
    test_set     = pd.read_csv("test.csv")
    

    labels = 'Survived'
    
    training_set, test_set = clean_data(training_set, test_set)
    
    training_features, training_labels =training_set, training_set.pop(labels)
    
    
    training_set.pop('Embarked')
    training_set.pop('Sex')
    
    tf_features = []
    for feature in training_features:
        if feature == 'Sex' or feature == 'Embarked':
            print("cool")
        else:
            tf_features.append(tf.feature_column.numeric_column(key = feature))
    classifier = model(tf_features)
    display(training_features)
    for i in range(100):
        print(training_features)
    classifier.train(input_fn = lambda:training_input_fn(training_features,training_labels, 10), steps=5000)
    

def training_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(buffer_size=1000).repeat(count=None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()


def remove_features(list, training_set,test_set):
    training_set = training_set.drop(labels=list, axis=1)
    test_set     = test_set.drop(labels=list, axis=1)
    return training_set, test_set


def fill_in_missing_ages(data_set):
    average_male_age   = np.mean(data_set.query("Sex != 'female'")['Age'])
    average_female_age = np.mean(data_set.query("Sex != 'male'")['Age'])
    
    males   =    data_set.index[data_set['Sex'] == 'male'].tolist()
    females =    data_set.index[data_set['Sex'] == 'female'].tolist()
    
    for i in range(len(data_set.count(axis=1))):
        if((data_set.loc[i,'Sex'] == 'male') and np.isnan(data_set.loc[i,'Age'])):
            data_set.loc[i,'Age'] = average_male_age
        elif((data_set.loc[i,'Sex'] == 'female') and np.isnan(data_set.loc[i,'Age'])):
            data_set.loc[i,'Age'] = average_female_age
    return data_set
    
def fix_missing_data(training_set, test_set):
    
    training_set = fill_in_missing_ages(training_set)
    test_set = fill_in_missing_ages(test_set)
    
    return training_set, test_set
    
def clean_data(training_set,test_set):
    bad_features = ['Name', 'Ticket','Cabin','PassengerId']
    
    
    training_set, test_set = remove_features(
        bad_features,
        training_set,
        test_set)
    
    training_set, test_set = fix_missing_data(training_set, test_set)
    one_hot_encode(training_set,test_set)
    
    #...
    
    return training_set, test_set

def encode_columns(training_set,test_set, columns):
    training_set.dropna()
    test_set.dropna()
    
    for i in columns:
        label_encoder = LabelEncoder()
        label_binarizer = LabelBinarizer()
        
        label_encoder.fit(training_set[i].tolist())
        encoded_train = label_encoder.transform(training_set[i].tolist())
        encoded_test  = label_encoder.transform(test_set[i].tolist())
       
        n_train = training_set.columns[training_set.columns.get_loc(i)]
        n_test = test_set.columns[test_set.columns.get_loc(i)]
        
        training_set.drop(n_train, axis=1, inplace=True)
        label_binarizer.fit(encoded_train.tolist())
        training_set[n_train] = label_binarizer.transform(encoded_train.tolist()).tolist()
        
        test_set.drop(n_test, axis=1, inplace=True)
        test_set[n_test] = label_binarizer.transform(encoded_test.tolist()).tolist()
       
        

def one_hot_encode(training_set, test_set):
    columns=['Sex','Embarked']
    encode_columns(training_set,test_set, columns=columns)
    
def model(training_features):
    return tf.estimator.DNNClassifier(
    feature_columns=training_features,
    hidden_units=[10,10],
    optimizer='Adam',
    activation_fn=tf.nn.relu,
    n_classes=2,
    )
    
main()   
    
    


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpcbf6shy5', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce200bcef0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


,Pclass,Age,SibSp,Parch,Fare
0,3,22.000000,1,0,7.2500
1,1,38.000000,1,0,71.2833
2,3,26.000000,0,0,7.9250
3,1,35.000000,1,0,53.1000
4,3,35.000000,0,0,8.0500
5,3,30.726645,0,0,8.4583
6,1,54.000000,0,0,51.8625
7,3,2.000000,3,1,21.0750
8,3,27.000000,0,2,11.1333
9,2,14.000000,1,0,30.0708


     Pclass        Age  SibSp  Parch      Fare
0         3  22.000000      1      0    7.2500
1         1  38.000000      1      0   71.2833
2         3  26.000000      0      0    7.9250
3         1  35.000000      1      0   53.1000
4         3  35.000000      0      0    8.0500
5         3  30.726645      0      0    8.4583
6         1  54.000000      0      0   51.8625
7         3   2.000000      3      1   21.0750
8         3  27.000000      0      2   11.1333
9         2  14.000000      1      0   30.0708
10        3   4.000000      1      1   16.7000
11        1  58.000000      0      0   26.5500
12        3  20.000000      0      0    8.0500
13        3  39.000000      1      5   31.2750
14        3  14.000000      0      0    7.8542
15        2  55.000000      0      0   16.0000
16        3   2.000000      4      1   29.1250
17        2  30.726645      0      0   13.0000
18        3  31.000000      1      0   18.0000
19        3  27.915709      0      0    7.2250
20        2  

     Pclass        Age  SibSp  Parch      Fare
0         3  22.000000      1      0    7.2500
1         1  38.000000      1      0   71.2833
2         3  26.000000      0      0    7.9250
3         1  35.000000      1      0   53.1000
4         3  35.000000      0      0    8.0500
5         3  30.726645      0      0    8.4583
6         1  54.000000      0      0   51.8625
7         3   2.000000      3      1   21.0750
8         3  27.000000      0      2   11.1333
9         2  14.000000      1      0   30.0708
10        3   4.000000      1      1   16.7000
11        1  58.000000      0      0   26.5500
12        3  20.000000      0      0    8.0500
13        3  39.000000      1      5   31.2750
14        3  14.000000      0      0    7.8542
15        2  55.000000      0      0   16.0000
16        3   2.000000      4      1   29.1250
17        2  30.726645      0      0   13.0000
18        3  31.000000      1      0   18.0000
19        3  27.915709      0      0    7.2250
20        2  

     Pclass        Age  SibSp  Parch      Fare
0         3  22.000000      1      0    7.2500
1         1  38.000000      1      0   71.2833
2         3  26.000000      0      0    7.9250
3         1  35.000000      1      0   53.1000
4         3  35.000000      0      0    8.0500
5         3  30.726645      0      0    8.4583
6         1  54.000000      0      0   51.8625
7         3   2.000000      3      1   21.0750
8         3  27.000000      0      2   11.1333
9         2  14.000000      1      0   30.0708
10        3   4.000000      1      1   16.7000
11        1  58.000000      0      0   26.5500
12        3  20.000000      0      0    8.0500
13        3  39.000000      1      5   31.2750
14        3  14.000000      0      0    7.8542
15        2  55.000000      0      0   16.0000
16        3   2.000000      4      1   29.1250
17        2  30.726645      0      0   13.0000
18        3  31.000000      1      0   18.0000
19        3  27.915709      0      0    7.2250
20        2  

[891 rows x 5 columns]
     Pclass        Age  SibSp  Parch      Fare
0         3  22.000000      1      0    7.2500
1         1  38.000000      1      0   71.2833
2         3  26.000000      0      0    7.9250
3         1  35.000000      1      0   53.1000
4         3  35.000000      0      0    8.0500
5         3  30.726645      0      0    8.4583
6         1  54.000000      0      0   51.8625
7         3   2.000000      3      1   21.0750
8         3  27.000000      0      2   11.1333
9         2  14.000000      1      0   30.0708
10        3   4.000000      1      1   16.7000
11        1  58.000000      0      0   26.5500
12        3  20.000000      0      0    8.0500
13        3  39.000000      1      5   31.2750
14        3  14.000000      0      0    7.8542
15        2  55.000000      0      0   16.0000
16        3   2.000000      4      1   29.1250
17        2  30.726645      0      0   13.0000
18        3  31.000000      1      0   18.0000
19        3  27.915709      0      0 

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpcbf6shy5/model.ckpt.
INFO:tensorflow:loss = 43.4132, step = 1
INFO:tensorflow:global_step/sec: 1156.03
INFO:tensorflow:loss = 4.37123, step = 101 (0.087 sec)
INFO:tensorflow:global_step/sec: 1932.54
INFO:tensorflow:loss = 7.91976, step = 201 (0.052 sec)
INFO:tensorflow:global_step/sec: 1875.07
INFO:tensorflow:loss = 4.63837, step = 301 (0.053 sec)
INFO:tensorflow:global_step/sec: 1871.32
INFO:tensorflow:loss = 5.64479, step = 401 (0.053 sec)
INFO:tensorflow:global_step/sec: 1601.33
INFO:tensorflow:loss = 9.76849, step = 501 (0.062 sec)
INFO:tensorflow:global_step/sec: 1794.02
INFO:tensorflow:loss = 6.18998, step = 601 (0.056 sec)
INFO:tensorflow:global_step/sec: 1730.35
INFO:tensorflow:loss = 3.81732, step = 701 (0.058 sec)
INFO:tensorflow:global_step/sec: 1764.6
INFO:tensorflow:loss = 8.93226, step = 801 (0.057 sec)
INFO:tensorflow:global_step/sec: 1686.31
INFO:tensorflow:loss = 5.5908, s